In [3]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [193]:
import pandas as pd
df = pd.read_excel('hotels.xlsx')
df.head()

,ID,Hotel name,Price(BAM),Hotel star rating,Distance,Customer rating,Rooms,Squares,City
0,1,Europe,139,5,350,8.3,1,25,Sarajevo
1,2,Europe,187,5,350,8.3,2,40,Sarajevo
2,3,Hills,255,5,10000,8.5,3,42,Sarajevo
3,4,Hills,141,5,10000,8.5,2,42,Sarajevo
4,5,Boutique,117,4,450,8.7,1,15,Sarajevo


In [120]:
df.City.unique()

array(['Sarajevo', 'Zagreb', 'Belgrade', 'Ljubljana'], dtype=object)

#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect Hotels for their preferences.

In [76]:
df.Distance.describe()

count      120.00000
mean      2353.75000
std       2437.27326
min        150.00000
25%        450.00000
50%       1550.00000
75%       3400.00000
max      10000.00000
Name: Distance, dtype: float64

In [14]:
df.Rooms.describe()

count    120.000000
mean       2.000000
std        0.916698
min        1.000000
25%        1.000000
50%        2.000000
75%        2.250000
max        4.000000
Name: Rooms, dtype: float64

In [6]:
df['City'].unique()

array(['Sarajevo', 'Zagreb', 'Belgrade', 'Ljubljana'], dtype=object)

In [5]:
df['Hotel name'].unique()

array(['Europe', 'Hills', 'Boutique', 'Radon plaza', 'Holiday inn',
       'Grand', 'Hayat', 'Brkic', 'Saraj', 'Old town', 'Ruža',
       'Riverside residence', 'Bristol', 'Opal home', 'Hollywood', 'Fala',
       'Zagreb', 'Best western', 'Forty two', 'Viena', 'Palace',
       'International', 'Garden', 'Esplanade', 'Double tree', 'Dubrovnik',
       'Academia', 'Rebro', 'Orient ekspres', 'The loop',
       'Public House Hotel\xa0', 'Envoy Hotel', 'Hilton Belgrade\xa0',
       'Hotel Bohemian Garni - Skadarlija', 'Mama Shelter Belgrade\xa0',
       'Hotel N\xa0', 'Hotel Rex\xa0', 'Room 25 Belgrade\xa0',
       'Garni Hotel Apart K\xa0', 'City Code Exclusive\xa0',
       'Hotel Avalon Garni\xa0', 'Hyatt', 'Square nine',
       'Metropol palace', 'Hotel Majestic', 'Saint Ten Hotel',
       'In Hotel ', 'Falkensteiner Hotel', 'Hotel Moskva',
       'Tullip Inn Putnik', 'Hotel Union', 'Hotel Elegance',
       'Hotel Emonec\xa0', 'Atelier Hotel\xa0', 'Adora Hotel',
       'Hotel Center Ljub

In [7]:
df['Hotel star rating'].unique()

array([5, 4, 3, 2])

In [10]:
df['Price(BAM)'].describe()

count    120.000000
mean     153.341667
std       79.444235
min       39.000000
25%       99.500000
50%      140.000000
75%      180.000000
max      474.000000
Name: Price(BAM), dtype: float64

### Here we build our AI system

In [194]:
import os,json,ast
import openai

In [195]:
folder_path = "/Users/taransingh/Desktop/Travel_Assist_AI/"
file_path = folder_path + "openapi_key.txt"

# Read the API key from the file
with open(file_path, "r") as f:
    api_key = f.read().strip()

# Set the API key for the openai library
openai.api_key = api_key

In [223]:
def initialize_conversation():
    """
    Returns a list [{"role": "system", "content": system_message}]
    """

    delimiter = "####"
    example_user_req = {'City': 'Sarajevo','Number of Rooms': 1,'Distance from city center':'Low', 'Hotel Star Ratings': '3','Budget in BAM': '60'}

    system_message = f"""
    
    You are an experienced Hotel guide and your goal is to recommend the best Hotel to a user.
    You need to ask relevant questions and understand the user profile and requirements by analysing the user's responses.
    You final objective is to fill the values for the different keys ('City','Number of Rooms','Distance from city centre','Hotel Star Ratings','Budget in BAM') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'City': 'values','Number of Rooms': values,'Distance from city centre': values,  'Hotel Star Ratings': 'values','Budget in BAM': 'values'}}
    The values for key 'City' should be either of 'Sarajevo','Zagreb','Belgrade','Ljubljana'.
    The values for key 'Number of Rooms' should be strictly below 5.
    The values for key 'Distance from city center' should be either 'Low'or 'High'.
    The values for key 'Hotel Star ratings' should be from 2 to 5.
    The values currently in the dictionary are only representative values.
    
    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'City' value needs to be either of 'Sarajevo','Zagreb','Belgrade','Ljubljana'. If the user says any other city, please mention that there are no hotels in that city that we can suggest.
    - 'Number of Rooms' value needs to be from 1 to 4. If the user says more than 4 rooms, please mention that there are a maximum of 4 rooms available.
    - 'Distance from city center' value needs to be either of either 'low' or 'high'.
    - 'Budget' value needs to be greater than or equal to 50 BAM. If the user says less than that, please mention that there are no hotels in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}
    
    
    To fill the dictionary, you need to have the following chain of thoughts:    
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary requirements for a hotel is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('City','Number of Rooms','Distance from city centre','Hotel Ratings','Budget in BAM') in the python dictionary and be confident of the values.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}
    
    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}
    
    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n
    
    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am on vacation with my family and we are in search of a hotel."
    Assistant: That's wonderful! Going on vacation with your spouse is a great way to create lasting memories. To help you find the perfect hotel, I'll need a bit more information: Which city are you in currently or planning to visit among the following options: (Sarajevo, Zagreb, Belgrade, Ljubljana)?"
    User: "I am in Sarajevo."
    Assistant: "Great! You're currently in Sarajevo. Do you wish to stay far or close to the city center?"
    User: "I'd stay nearby the City Center"
    Assistant: "Thank you for the information. How many rooms do you need for your stay?"
    User: "We will be staying in 1 room"
    Assistant:"Could you kindly let me know your budget per night in BAM? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 60 BAM"
    Assistant:"Alright lastly please let me know if you prefer Hotel Star ratings from 2 to 5?"
    User: "my max budget is 60 BAM"
    Assistant: "{example_user_req}"
    {delimiter}
    {delimiter}
    Make sure to provide the cities 'Sarajevo','Zagreb','Belgrade','Ljubljana', in your first question for the user to chose from.
    Ask the questions in strict order:
    1. City confirmation
    2. Far or Nearby City Center confirmation
    3. Number of rooms confirmation
    4. Budget confirmation
    5. Hotel Star Ratings confirmation
    
    Dont take user input on an empty question , user should be able to answer 1 of the 5 above, in order.
    {delimiter}
    
    
    Start with a short welcome message, greet the user well and figure out the preferences in the following messages.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [224]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    \n    You are an experienced Hotel guide and your goal is to recommend the best Hotel to a user.\n    You need to ask relevant questions and understand the user profile and requirements by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'City\',\'Number of Rooms\',\'Distance from city centre\',\'Hotel Star Ratings\',\'Budget in BAM\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this {\'City\': \'values\',\'Number of Rooms\': values,\'Distance from city centre\': values,  \'Hotel Star Ratings\': \'values\',\'Budget in BAM\': \'values\'}\n    The values for key \'City\' should be either of \'Sarajevo\',\'Zagreb\',\'Belgrade\',\'Ljubljana\'.\n    The values for key \'Number of Rooms\' should be strictly below 5.\n    The values for key \'Distance from city center\' should be either \'Low\'or

In [225]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, 
        max_tokens = 1900
    )
    return response.choices[0].message["content"]

In [226]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Assistant: "Hello! Welcome to our hotel recommendation service. How can I assist you today?"


In [227]:
debug_user_input = "Hi, I am Taran. I am looking for a hotel in Sarajevo."

In [228]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

That's wonderful, Taran! Sarajevo is a beautiful city. To help you find the perfect hotel, I'll need a bit more information. Do you wish to stay far or close to the city center?


In [229]:
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [203]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - City
   - Number of Rooms
   - Distance from City Center
   - Hotel Star Ratings
   - Budget

In [230]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: City, Number of Rooms, Distance from city center, Hotel Star Ratings, Budget in BAM
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for key 'City' should be either 'Sarajevo','Zagreb','Belgrade','Ljubljana'.
    The values for key 'Number of Rooms' should be strictly below 5.
    The values for key 'Distance from city center should be 'low' or 'high'.
    The values for key 'Hotel Star Ratings' should be from 2 to 5.
    The value for 'budget' should be a numerical value extracted from the user's response in BAM currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """


    confirmation = openai.Completion.create(
                                    model="text-davinci-003",
                                    prompt = prompt,
                                    temperature=0)


    return confirmation["choices"][0]["text"]

In [231]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)


No


In [232]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:

{{'City': 'Sarajevo','Number of Rooms': 1,'Distance from city centre': 'low',  'Hotel Star Ratings': 3,'Budget in BAM': 90}}

Is there anything else I can assist you with?
"""
#Note that you are using double curly braces

In [233]:
intent_confirmation_layer(debug_response_assistant_n)

'\nYes'

In [234]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'City': 'Sarajevo','Number of Rooms': 1,'Distance from city centre': 'low',  'Hotel Star Ratings': 3,'Budget in BAM': 90}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - City: 'Sarajevo'- Number of Rooms: 1 - Distance from city centre: 'low' - Hotel Star Ratings: 3 - Budget in BAM: 90
            output: {{'City': 'Sarajevo','Number of Rooms': 1,'Distance from city centre': 'low',  'Hotel Star Ratings': 3,'Budget in BAM': 90}}

            input: {{'City': 'Zagreb','Number of Rooms': 2,'Distance from city centre': 'high',  'Hotel Star Ratings': 5,'Budget in BAM': 190}}
            output: {{'City': 'Zagreb','Number of Rooms': 2,'Distance from city centre': 'high',  'Hotel Star Ratings': 5,'Budget in BAM': 190}}

            input: Here is your user profile 'City': 'Sarajevo','Number of Rooms': 1,'Distance from city centre': 'low',  'Hotel Star Ratings': 4,'Budget in BAM': 120
            output: {{'City': 'Sarajevo','Number of Rooms': 1,'Distance from city centre': 'low',  'Hotel Star Ratings': 4,'Budget in BAM': 120}}
            {delimiter}

            Here is the input {response}

            """
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens = 2000
        # temperature=0.3,
        # top_p=0.4
    )
    return response["choices"][0]["text"]

In [209]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 60 BAM, I will consider this while recommending suitable hotels for your stay in Sarajevo City.
Here is the final recommendation of your Hotel:

- City: 'Sarajevo'
- Number of Rooms: 1
- Distance from city center: 'low'
- Hotel Star Ratings: 4
- Budget in BAM: 60

Please note that these specifications are based on your preferences within your budget. Let me know if there's anything else I can assist you with!"""

In [235]:
def hotel_map_layer(df):
    keys = ["City", "Number of Rooms", "Distance from city center", "Hotel Star ratings", "Budget in BAM"]

    hotels_list = []

    for index, row in df.iterrows():
        # Create a dictionary for each row
        hotel_req = {
            "City": row["City"] if row["City"] in ['Sarajevo', 'Zagreb', 'Belgrade', 'Ljubljana'] else None,
            "Number of Rooms": row["Rooms"],
            "Distance from city center": "high" if row["Distance"] > 2500 else "low",  # You can set your own threshold
            "Hotel Star Ratings": row["Hotel star rating"],  # You might want to validate this against available star ratings
            "Budget in BAM": row["Price(BAM)"]  # You might want to validate this against a budget range
        }
        # Append the dictionary to the list
        hotels_list.append(hotel_req)

    # Add the hotel_req_list as a new column to the DataFrame
    df = df.assign(Hotels=hotels_list)

    return df


df = hotel_map_layer(df)


In [236]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [237]:
import pandas as pd
import json

def compare_hotels_with_user(user_req_string, df):
    
    df = df
    # Step 1: Filter hotel_req_list based on the user's city requirement
    df_hotels = df[df['Hotels'].apply(lambda x: x['City'].lower() == user_req_string['city'].lower())]

    # Step 2: Filter hotel_req_list based on the number of rooms requirement
    df_hotels = df_hotels[df_hotels['Hotels'].apply(lambda x: x['Number of Rooms'] >= user_req['number of rooms'])]

    # Step 3: Filter hotel_req_list based on the budget requirement
    df_hotels = df_hotels[df_hotels['Hotels'].apply(lambda x: x['Budget in BAM'] <= user_req['budget in bam'])]

    # Step 4: Further shortlist based on distance from city center and hotel star ratings
    df_hotels = df_hotels[df_hotels['Hotels'].apply(
        lambda x: x['Distance from city center'].lower() == user_req['distance from city centre'].lower()
                    and x['Hotel Star Ratings'] == user_req['hotel star ratings']
    )]

    # Step 5: Calculate a score for each hotel based on matching user requirements
    def calculate_score(row):
        score = 0
        if row['Hotels']['City'].lower() == user_req['city'].lower():
            score += 1
        if row['Hotels']['Number of Rooms'] >= user_req['number of rooms']:
            score += 1
        if row['Hotels']['Budget in BAM'] <= user_req['budget in bam']:
            score += 1
        if (row['Hotels']['Distance from city center'].lower() == user_req['distance from city centre'].lower() and
            row['Hotels']['Hotel Star Ratings'] == user_req['hotel star ratings']):
            score += 1
        return score

    df_hotels['Score'] = df_hotels.apply(calculate_score, axis=1)

    # Step 6: Sort hotels based on their scores in descending order
    sorted_df = df_hotels.sort_values(by='Score', ascending=False)

    # Step 7: Return the top 3 hotel recommendations as a JSON-formatted string
    top_recommendations = sorted_df.head(3)[['Hotel name','Hotels']].to_dict(orient='records')
    json_recommendations = json.dumps(top_recommendations, indent=4)

    return json_recommendations




In [238]:
import json

def format_recommendations_to_user(top_hotels):
    # Load the JSON string into a Python list of dictionaries
    recommendations = json.loads(top_hotels)

    if not recommendations:
        return "No hotels matching your criteria were found."

    formatted_recommendations = "Here is a list of hotels, Please go ahead and book, Thanks for choosing HotelFinders:\n"

    for idx, hotel in enumerate(recommendations, start=1):
        formatted_recommendations += f"{idx}. {hotel['Hotel name']}, located in City {hotel['Hotels']['City']}.\n"
        formatted_recommendations += f"   Number of Rooms: {hotel['Hotels']['Number of Rooms']}\n"
        formatted_recommendations += f"   Distance from city center: {hotel['Hotels']['Distance from city center']}\n"
        formatted_recommendations += f"   Hotel Star Ratings: {hotel['Hotels']['Hotel Star Ratings']}\n"
        formatted_recommendations += f"   Budget in BAM: {hotel['Hotels']['Budget in BAM']}\n"

    return formatted_recommendations

# Example usage:
formatted_output = format_recommendations_to_user(top_hotels)


In [215]:
import pandas as pd
import json

def dialogue_system():
    # Read the DataFrame from a CSV file (assuming the file is named 'hotels_data.csv')
    

    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_hotels = None
    moderation = None  # Initialize moderation variable
    user_input = ''

    while user_input != 'exit':
        if moderation == 'Flagged':
            print("Sorry, Restart the conversation please.")
            break

        if top_hotels is None:
            user_input = input("User: ")  # Capture user input
            conversation.append({'role': 'user', 'content': user_input})
            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, Restart the conversation please.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, Restart the conversation please.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(confirmation)
                if moderation == 'Flagged':
                    print("Sorry, Restart the conversation please.")
                    break

                print('\n' + response + '\n')
                user_req = extract_dictionary_from_string(response)
                print(user_req)
                print("Thank you for providing all the information. Kindly wait, while I fetch the Hotels: \n")
                top_hotels = compare_hotels_with_user(user_req, df)
                output = format_recommendations_to_user(top_hotels)

                print(output)
                break

# Usage example:
result = dialogue_system()
print(result)


Assistant: "Hello! Welcome to our Hotel recommendation service. How can I assist you today?"

User: Hi I need hotel for vacation

That's wonderful! Going on vacation is always exciting. To help you find the perfect hotel, I'll need a bit more information. Which city are you currently in or planning to visit among the following options: Sarajevo, Zagreb, Belgrade, Ljubljana?



No

User: Belgrade

Great! You're currently in Belgrade. Do you wish to stay far or close to the city center?



No

User: close

Thank you for the information. How many rooms do you need for your stay in Belgrade?



No

User: 1

Could you kindly let me know your budget per night in BAM? This will help me find options that fit within your price range while meeting the specified requirements.



No

User: 130

{'City': 'Belgrade', 'Number of Rooms': 1, 'Distance from city centre': 'Close', 'Hotel Star Ratings': 'values','Budget in BAM': 130}



No

User: okay

Great! I have updated the information based on your r

In [239]:
import pandas as pd
import json

def dialogue_system():
    # Read the DataFrame from a CSV file (assuming the file is named 'hotels_data.csv')
    

    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_hotels = None
    moderation = None  # Initialize moderation variable
    user_input = ''

    while user_input != 'exit':
        if moderation == 'Flagged':
            print("Sorry, Restart the conversation please.")
            break

        if top_hotels is None:
            user_input = input("User: ")  # Capture user input
            conversation.append({'role': 'user', 'content': user_input})
            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, Restart the conversation please.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, Restart the conversation please.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(confirmation)
                if moderation == 'Flagged':
                    print("Sorry, Restart the conversation please.")
                    break

                print('\n' + response + '\n')
                user_req = extract_dictionary_from_string(response)
                print(user_req)
                print("Thank you for providing all the information. Kindly wait, while I fetch the Hotels: \n")
                top_hotels = compare_hotels_with_user(user_req, df)
                output = format_recommendations_to_user(top_hotels)

                print(output)
                break

# Usage example:
result = dialogue_system()
print(result)


Assistant: "Hello! Welcome to our hotel recommendation service. How can I assist you today?"

User: I need hotel for office visit

That's great! Finding the right hotel for your office visit is important. To help you with that, I'll need some information from you. Which city are you planning to visit among the following options: Sarajevo, Zagreb, Belgrade, Ljubljana?



No

User: Belgrade

Great! You're planning to visit Belgrade. Do you wish to stay far or close to the city center?



No

User: close

Thank you for the information. How many rooms do you need for your stay in Belgrade?



No

User: 1

Thank you for letting me know. Could you kindly let me know your budget per night in BAM? This will help me find options that fit within your price range while meeting the specified requirements.



No

User: 140

Thank you for providing your budget. Lastly, please let me know if you have any preference for hotel star ratings. Do you prefer hotels with star ratings from 2 to 5?



No

Use